In [1]:
!pip install transformers
!pip install imageio
!pip install accelerate
!pip install sentencepiece
!pip install protobuf
import datetime
import os
import imageio
import numpy as np
import torch
import torch.nn.functional as F
from PIL import Image
from transformers import MistralForCausalLM, AutoTokenizer, AutoConfig

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 143.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.6/402.6 kB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.1/775.1 kB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 116.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 145.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.9/176.9 kB 37.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.4.0
    Uninstalling fsspec-2023.4.0:
      Successfully uninstalled fsspec-2023.4.0

[notice] A new release of pip is

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [11]:
device_0 = "cuda:0"
model_folder = "valine/OpenPirate"  # name of the model
image_output_folder = "/workspace"

In [4]:
#这段代码定义了一个名为 main 的函数，用于加载一个预训练的语言模型和分词器，生成嵌入结果，并调用前面定义的函数来可视化这些嵌入结果
#该 main 函数执行以下步骤：
  #加载预训练模型的配置、模型和分词器
  #定义一个字符串 probe_string，用于生成嵌入结果
  #计算 probe_string 的嵌入结果，并将其存储在 probe_results 列表中
  #调用 plot_embedding_flow 函数，对嵌入结果进行可视化，并生成一个 GIF 动图

def main():
    #从 model_folder 中加载预训练模型的配置 config
    config = AutoConfig.from_pretrained(model_folder)
    
    #加载一个名为 MistralForCausalLM 的预训练语言模型 mistral
      #model_folder：模型存储的文件夹路径。
      #torch_dtype=torch.float16：指定模型使用 16 位浮点数
      #device_map=device_0：指定模型加载到的设备（例如 GPU）
      #use_flash_attention_2=False：禁用 Flash Attention 2
      #config=config：使用之前加载的配置
    mistral = MistralForCausalLM.from_pretrained(
        model_folder,
        torch_dtype=torch.float16,
        device_map=device_0,
        use_flash_attention_2=False,
        config=config, )

    #从 model_folder 中加载预训练分词器 tokenizer. trust_remote_code=True：信任远程代码以允许自定义分词器的代码执行
    tokenizer = AutoTokenizer.from_pretrained(
        model_folder, trust_remote_code=True)

    # The last token of this string will be used to generate the image
    # 定义一个用于生成嵌入结果的字符串 probe_string
    probe_string = "Put a sample string here"

    # Probe results is an array so that you can plot the changes to the
    # output over time. The plot_embedding_flow will generate an animated gif.
    # Call compute_model_output multiple times and append the results to
    # probe_results.
    # 初始化一个空列表 probe_results，用于存储不同时间步的嵌入结果
     # 调用 compute_model_output 函数计算 probe_string 的嵌入结果，并将其添加到 probe_results 中
     # mistral：预训练语言模型
     # tokenizer：预训练分词器
     # probe_string：用于生成嵌入结果的字符串
    probe_results = []
    probe_result = compute_model_output(mistral, tokenizer, probe_string)
    probe_results.append(probe_result)

    # 调用 plot_embedding_flow 函数，对生成的嵌入结果进行可视化，并生成一个 GIF 动图
    # probe_results：包含所有嵌入结果的列表
    plot_embedding_flow(probe_results)

In [5]:
#这段代码定义了一个名为 compute_model_output 的函数，该函数用于计算一个基于预训练语言模型的输出

def compute_model_output(base_model, tokenizer, ground_truth):
    # 使用 no_grad() 上下文管理器，在计算过程中不计算梯度，减少内存使用
    with torch.no_grad():
        #用于存储每层的输出
        layer_output = [] 

        # 使用 tokenizer 对 ground_truth 进行编码，生成 PyTorch 张量, 并将结果存储在 input_ids 中
        encoding = tokenizer(ground_truth, return_tensors="pt")
        input_ids = encoding['input_ids'].to(device_0)

        # 获取输入的词嵌入（embedding）,使用模型的嵌入层 embed_tokens 将 input_ids 转换为词嵌入 hidden_states
        hidden_states = base_model.model.embed_tokens(input_ids)

        # 获取序列长度和批次大小
        sequence_length = hidden_states.shape[1]
        batch_size = hidden_states.shape[0]

        # 创建一个表示位置编码的张量 position_ids，其形状为 (1, sequence_length), 使用 expand 方法将其扩展为 (batch_size, sequence_length)
        position_ids = torch.arange(sequence_length, device=device_0).unsqueeze(0)
        position_ids = position_ids.expand(batch_size, -1)

        # 创建一个上三角矩阵 attention_mask，用于掩码未来的 token，其形状为 (sequence_length, sequence_length)
        # 使用 unsqueeze 方法将其形状扩展为 (1, 1, sequence_length, sequence_length)
        attention_mask = torch.triu(torch.full(
            (sequence_length, sequence_length), float('-inf')), diagonal=1)
        attention_mask = attention_mask.to(device_0)
        attention_mask = attention_mask.unsqueeze(0).unsqueeze(0)

        # 遍历模型的每一层，计算每层的输出
        # 调用每层的 forward 方法，传入 hidden_states、attention_mask 和 position_ids
        # 将每层的输出 hidden_states 添加到 layer_output 列表中
        for layer in base_model.model.layers:
            output = layer(hidden_states,
                           attention_mask=attention_mask,
                           position_ids=position_ids,
                           output_attentions=True)
            hidden_states = output[0]
            layer_output.append(hidden_states)
        return layer_output  #返回存储所有层输出的列表 layer_output


In [6]:
#这段代码定义了一个函数 vectorized_get_color_rgb，它将一个取值在 [0, max_value] 区间的张量转换为 RGB 颜色表示
#这段代码实现了将归一化的 HSV 颜色值转换为 RGB 颜色值的功能

def vectorized_get_color_rgb(value_tensor, max_value=1.0):
    #h是归一化后的 value_tensor，范围为 [0, 1]
    h = (value_tensor * 1.0) / max_value  
    # s 和 v 被初始化为与 h 形状相同且值全为1的张量，表示饱和度和亮度均为1
    s = torch.ones_like(h)  
    v = torch.ones_like(h)
    
    # c 是颜色值，根据 HSV 色彩模型中的公式 c = v * s 计算
    # x 是中间值，用于计算 RGB 值
    # m 是最小值，用于计算最终的 RGB 值
    c = v * s
    x = c * (1 - torch.abs((h * 6) % 2 - 1))
    m = v - c
    
    # h1 用于确定颜色所在的区间
    h1 = (h * 6).int()
    
    #使用 torch.stack 将三个通道（R、G、B）的值合并在一起，生成一个 RGB 张量
    #对每个通道，根据 h1 的值确定使用 c、x 还是 0：
     #红色通道：当 h1 为 0 或 5 时，值为 c；当 h1 为 1 或 4 时，值为 x；否则为 0
     #绿色通道：当 h1 为 1 或 2 时，值为 c；当 h1 为 0 或 3 时，值为 x；否则为 0
     #蓝色通道：当 h1 为 3 或 4 时，值为 c；当 h1 为 2 或 5 时，值为 x；否则为 0
    #将 m 加到每个通道上，调整亮度
    rgb = torch.stack((
        torch.where((h1 == 0) | (h1 == 5), c, torch.where((h1 == 1) | (h1 == 4), x, 0)),
        torch.where((h1 == 1) | (h1 == 2), c, torch.where((h1 == 0) | (h1 == 3), x, 0)),
        torch.where((h1 == 3) | (h1 == 4), c, torch.where((h1 == 2) | (h1 == 5), x, 0)),
    ), dim=-1) + m.unsqueeze(-1)

    return rgb  #返回计算得到的 RGB 张量

In [7]:
#这段代码定义了一个函数 generate_filename，用于生成包含当前时间戳的文件名
#prefix：文件名前缀  extension：文件扩展名

def generate_filename(prefix, extension): 
    #获取当前日期和时间
    current_time = datetime.datetime.now()
    
    #将 current_time 格式化为 "年-月-日_时-分-秒" 格式
    formatted_time = current_time.strftime("%Y%m%d_%H%M%S")
    
    #生成的文件名格式为 "前缀_格式化时间.扩展名"
    filename = f"{prefix}_{formatted_time}.{extension}"
    return filename

In [8]:
#这段代码定义了一个名为 plot_layers 的函数，该函数将多个层的嵌入结果可视化，并生成一个包含这些结果的 GIF 动图
#该函数将多个层的嵌入结果转换为 RGB 图像，并将这些图像生成一个 GIF 动图
#在生成过程中，对数据进行归一化处理，并使用填充和分割技术将张量转换为图像
#最终返回生成的 GIF 动图的文件路径

#定义一个函数 plot_layers，接收四个参数：
  #all_words：一个包含所有层嵌入结果的列表。
  #title：生成文件的标题前缀。
  #file_path：保存图像和 GIF 的文件路径。
  #normalize：一个布尔值，指示是否对数据进行归一化处理，默认值为 True
def plot_layers(all_words, title, file_path, normalize=True):
    #获取序列长度 sequence_length，并初始化一个空列表 paths 用于存储每张图像的路径
    sequence_length = all_words[0].shape[1]
    paths = []
    
    #将 all_words 中的所有张量沿第一个维度拼接，形成一个大的张量 all_words_cat
    #计算拼接后的张量的全局最小值 global_min_val 和全局最大值 global_max_val
    #计算全局均值 global_mean 和全局方差 global_var，并将方差乘以 25 以扩展其范围
    all_words_cat = torch.cat(all_words, dim=0)
    global_min_val = torch.min(all_words_cat)
    global_max_val = torch.max(all_words_cat)
    global_mean = torch.mean(all_words_cat)
    global_var = torch.var(all_words_cat) * 25

    #如果需要归一化处理，则设置 min_val 和 max_val 为均值减去/加上方差的范围
    #否则，设置 min_val 和 max_val 为全局最小值和最大值
    if normalize:
        min_val = global_mean - global_var
        max_val = global_mean + global_var
    else:
        min_val = global_min_val
        max_val = global_max_val
        
    #外层 for 循环遍历序列的每个时间步 i
    for i in range(sequence_length):
        
        #初始化一个空列表 list_of_tensors，用于存储当前时间步的所有嵌入结果。
        list_of_tensors = []
        
        #内层 for 循环遍历 all_words 中的每个张量，从每个张量中提取第 i 个时间步的嵌入结果，并添加到 list_of_tensors 中
        for tensor in all_words:
            list_of_tensors.append(tensor[:, i, :])
        
        # 将 list_of_tensors 中的所有张量沿第一个维度拼接，形成一个大的张量 full_tensor
        # 设置图像的高度 height 为 512 像素
        # Step 1: Concatenate tensors along width
        full_tensor = torch.cat(list_of_tensors, dim=0)  # Shape: [1, 4096 * 31]
        height = 512
        
        #将 full_tensor 沿第二个维度分割成多个块，每个块的宽度为 height
        #使用 F.pad 函数对每个块进行填充，使其宽度达到 height，填充值为 max_val
        #将这些填充后的块沿第一个维度拼接，形成一个新的张量 reshaped_tensor，并取其绝对值
        tensor_split = [F.pad(t, (0, max(0, height - t.shape[1])),
                              'constant', max_val.item()) for t in
                        torch.split(full_tensor, height, dim=1)]
        reshaped_tensor = torch.cat(tensor_split, dim=0)
        reshaped_tensor = torch.abs(reshaped_tensor)

        # Normalize data
        # 对 reshaped_tensor 进行归一化处理，使其值在 [0, 1] 范围内
        # 调用 vectorized_get_color_rgb 函数，将归一化后的数据转换为 RGB 颜色表示
        normalized_data = (reshaped_tensor - min_val) / (max_val - min_val)
        color_tensor = vectorized_get_color_rgb(normalized_data)

        # Generate image
        # 将 color_tensor 转换为 NumPy 数组，并将其值缩放到 [0, 255] 范围，数据类型转换为 uint8
        #使用 Image.fromarray 函数将 NumPy 数组转换为图像对象 image
        array = (color_tensor.cpu().numpy() * 255).astype(np.uint8)
        image = Image.fromarray(array, 'RGB')

        # Save the image
        #生成临时文件名 tmp_name。
        #调用 generate_filename 函数生成图像文件名 filename。
        #将图像文件保存到指定路径 full_path。
        #将 full_path 添加到 paths 列表中。
        tmp_name = "raw_values_tmp" + str(i)
        filename = title + "_" + generate_filename(tmp_name + str(i), "png")
        full_path = os.path.join(file_path, filename)
        image.save(full_path)
        paths.append(full_path)

    # Create gif from images
    # 生成 GIF 文件名 filename
    # 使用 imageio.get_writer 函数创建一个 GIF 写入对象 writer，设置帧率 fps 为 15，循环次数 loop 为 0
    # 遍历 paths 列表，读取每个图像文件，并将其添加到 GIF 中
    filename = title + "_" + generate_filename("layers", "gif")
    gif_path = os.path.join(file_path, filename)
    with imageio.get_writer(gif_path, mode='I', fps=15, loop=0) as writer:
        for filename in paths:
            image = imageio.imread(filename)
            writer.append_data(image)

    # Remove temporary files
    # 遍历 paths 列表，删除所有临时图像文件
    for filename in paths:
        os.remove(filename)

    # open_image(gif_path)
    # 返回生成的 GIF 文件路径 gif_path
    return gif_path

In [9]:
#这段代码定义了一个名为 plot_embedding_flow 的函数，用于处理和可视化多个层的嵌入结果
#该函数从 probe_results 中提取每一层的嵌入结果，并将其组织成一个新的张量列表
#通过 plot_layers 函数将这些嵌入结果可视化，并返回生成图像的路径

#定义一个函数 plot_embedding_flow，接收一个参数 probe_results，该参数包含多个层的嵌入结果
def plot_embedding_flow(probe_results):
    #计算 probe_results 中每个结果包含的层数
    layer_count = len(probe_results[0])
    
    #初始化一个空列表 layer_embeddings，用于存储每一层的嵌入结果
    layer_embeddings = []
    
    #外层 for 循环遍历所有层
    for l_index in range(layer_count):
        
        #初始化一个空列表 sequence_embedding，用于存储当前层的所有嵌入结果
        sequence_embedding = []
        
        #内层 for 循环遍历 probe_results 中的每个结果
        #从 probe_result 中提取当前层 l_index 的最后一个时间步的嵌入结果，并将其添加到 sequence_embedding 中
        for probe_result in probe_results:
            embedding = probe_result[l_index][:, -1, :]
            sequence_embedding.append(embedding)
            
        #将 sequence_embedding 列表堆叠为一个张量 layer_embedding，其维度为 (序列长度, 批次大小, 嵌入维度)
        layer_embedding = torch.stack(sequence_embedding, dim=1)
        layer_embeddings.append(layer_embedding)
    
    #调用 plot_layers 函数对当前进度进行可视化
      #layer_embeddings：包含所有层嵌入结果的列表
      #"probe_results"：用于标记图像的标签
      #image_output_folder：图像输出文件夹的路径
    # Plot current progress
    path = plot_layers(layer_embeddings, "probe_results", image_output_folder)
    return path #返回可视化图像的保存路径



In [12]:
if __name__ == "__main__":
    main()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/tmp/ipykernel_229/71677521.py:90: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)
